In [1]:
import mxnet as mx

In [2]:
mnist = mx.test_utils.get_mnist()
mx.random.seed(1)
batch_size = 32
ctx = mx.gpu()
nd_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size)

In [6]:
data = mx.sym.Variable('data')
conv1= mx.symbol.Convolution(data = data, name='conv1', num_filter=32, kernel=(3,3), stride=(2,2))
bn1 = mx.symbol.BatchNorm(data = conv1, name="bn1")
act1 = mx.symbol.Activation(data = bn1, name='relu1', act_type="relu")
mp1 = mx.symbol.Pooling(data = act1, name = 'mp1', kernel=(2,2), stride=(2,2), pool_type='max')

conv2= mx.symbol.Convolution(data = mp1, name='conv2', num_filter=32, kernel=(3,3), stride=(2,2))
bn2 = mx.symbol.BatchNorm(data = conv2, name="bn2")
act2 = mx.symbol.Activation(data = bn2, name='relu2', act_type="relu")
mp2 = mx.symbol.Pooling(data = act2, name = 'mp2', kernel=(2,2), stride=(2,2), pool_type='max')

fl = mx.symbol.Flatten(data = mp2, name="flatten")
fc2 = mx.symbol.FullyConnected(data = fl, name='fc2', num_hidden=10)
softmax = mx.symbol.SoftmaxOutput(data = fc2, name = 'softmax')
mod = mx.mod.Module(softmax)  # create a module by given a Symbol

In [7]:
# mod.bind(data_shapes=nd_iter.provide_data,
#          label_shapes=nd_iter.provide_label) # create memory by given input shapes
# mod.init_params() # initial parameters with the default random initializer

In [8]:
mod.fit(nd_iter, num_epoch=10) # train
mod.predict(nd_iter) # predict on new data


[[1.51827175e-03 3.75562850e-05 4.08565771e-04 ... 1.76200716e-04
  2.09606532e-03 1.40904053e-03]
 [9.99950528e-01 1.79718924e-07 7.49523770e-06 ... 1.34945294e-05
  5.62577497e-06 4.28289377e-06]
 [1.05948027e-04 3.55961956e-02 5.65566728e-03 ... 6.39976040e-02
  2.60407658e-04 2.31357361e-03]
 ...
 [2.06898003e-05 3.86129977e-05 6.13283992e-05 ... 2.59731692e-04
  7.45705256e-05 1.56577618e-03]
 [8.47095326e-02 1.28737956e-04 6.06133690e-05 ... 3.17675695e-05
  8.27927329e-03 7.11440734e-05]
 [1.63099077e-03 5.72913405e-06 1.20691366e-04 ... 3.23227978e-05
  9.75111365e-01 1.94353294e-02]]
<NDArray 60000x10 @cpu(0)>

In [9]:
mod.save_checkpoint('mod', 0)

In [10]:
sym, arg_params, aux_params = mx.model.load_checkpoint('mod', 0)

In [11]:
import logging
nd_iter.reset()
qsym, qarg_params, qaux_params = mx.contrib.quantization.quantize_model(sym=sym, arg_params=arg_params, aux_params=aux_params,
                                                ctx=ctx, calib_data=nd_iter, logger=logging)

In [9]:
mod = mx.mod.Module(qsym, context=ctx)

In [ ]:
nd_iter.reset()
mod.bind(for_training=False,
         data_shapes=nd_iter.provide_data,
         label_shapes=nd_iter.provide_label)

In [ ]:
mod.set_params(arg_params, aux_params)